# Events

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
import polars as pl
from discontinuitypy.datasets import IDsDataset
from beforerr.polars import pl_norm
from datetime import timedelta
from space_analysis.utils.speasy import Variables
from rich import print

In [53]:
from utils import PSPConfig, get_timerange

In [54]:
enc = 2
psp_timerange, earth_timerange = get_timerange(enc)

In [76]:
PSPConfig(
    enc = enc,
    timerange=psp_timerange,
    instr_p = 'spi'
).get_and_process_data().export()

Q: What is the time scale for change in magnetic field, density and velocity?

- Different dataset comes with different time resolution.
- Mangnetic field data has the highest time resolution.

method 1: directly interpolate density and velocity into magnetic field data respectively
method 2: interpolate density into velocity data first

### Process

In [59]:
psp_ids_dataset = (
    get_and_process_data(mag_dataset, mag_parameters, timerange, tau, ts, p_vars=p_vars)
    .find_events()
    .update_candidates_with_plasma_data()
    .export(fname)
)

19-Feb-24 16:05:49: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

## Wind dataset


In [16]:
ts = timedelta(seconds=1 / 11)
tau = timedelta(seconds=30)

mag_dataset = "WI_H2_MFI"
mag_parameters = ["BGSE"]

plasma_dataset="WI_PM_3DP"
plasma_parameters=["P_DENS", "P_VELS", "P_TEMP"] # Proton velocity vector (GSE)

# plasma_dataset="WI_K0_SWE"
# plasma_parameters=["Np", "V_GSM", "THERMAL_SPD"]

timerange = validate(earth_timerange)

fname = f"{dir}/wind_ids_dataset_{enc}.{fmt}"

In [ ]:

def get_and_process_data(
    mag_dataset,
    mag_parameters,
    timerange,
    tau,
    ts,
    p_vars: Variables = None,
    interpolate=True,
    every=timedelta(seconds=0.2),
):
    # define variables
    mag_vars = Variables(
        dataset=mag_dataset,
        parameters=mag_parameters,
        timerange=timerange,
    ).retrieve_data()

    p_vars.timerange = timerange
    p_vars.check_products()
    p_vars.retrieve_data()

    # get column names
    bcols = mag_vars.data[0].columns
    density_col = p_vars.data[0].columns[0]
    vec_cols = p_vars.data[1].columns
    temperature_col = p_vars.data[2].columns[0]

    # get data
    mag_data = mag_vars.to_polars().unique("time")
    plasma_data = (
        p_vars.to_polars()
        .with_columns(plasma_speed=pl_norm(vec_cols))
        .rename({density_col: "n"})
        .unique("time")
        # .drop_nulls()
    )
    # process temperature data
    if p_vars.data[2].unit == "km/s":
        plasma_data = plasma_data.pipe(df_thermal_spd2temp, temperature_col)
    else:
        plasma_data = plasma_data.rename({temperature_col: "plasma_temperature"})

    if interpolate:
        plasma_data = (
            plasma_data.with_columns(pl.col("n").fill_nan(None))
            .pipe(resample, every=every)
            .collect()
            .sort("time")
            .upsample("time", every=every)
            .interpolate()
            .lazy()
        )

    return IDsDataset(
        mag_data=mag_data,
        plasma_data=plasma_data,
        tau=tau,
        ts=ts,
        bcols=bcols,
        vec_cols=vec_cols,
        density_col="n",
        speed_col="plasma_speed",
        temperature_col="plasma_temperature",
    )

In [17]:
wind_ids_dataset = get_and_process_data(
    mag_dataset, mag_parameters, plasma_dataset, plasma_parameters, timerange, tau, ts
).find_events().update_candidates_with_plasma_data().export(fname)

11-Feb-24 12:50:50: Using pycdfpp
11-Feb-24 12:51:00: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 3:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [18]:
import hvplot.polars

In [19]:
wind_ids_dataset.plot()

2024-02-11 12:51:03.398 | INFO     | discontinuitypy.datasets:overview_plot:172 - Overview plot: 2020-01-29 23:40:15 - 2020-01-29 23:40:45
2024-02-11 12:51:03.398 | DEBUG    | discontinuitypy.datasets:log_event_change:79 - CHANGE INFO
        n.change: 0.4995388984680176
        v.ion.change: 7.461979130688064
        T.change: 0.236236572265625
        v.Alfven.change: -7.14332788724645
        v.ion.change.l: 0.392388073545078
        v.Alfven.change.l: 40.50811492639099
        


:Layout
   .Overlay.I   :Overlay
      .NdOverlay.I :NdOverlay   [B comp]
         :Curve   [time]   (B)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.II  :Overlay
      .Curve.I   :Curve   [time]   (plasma_density)
      .Scatter.I :Scatter   [time]   (plasma_density)
      .VLine.I   :VLine   [x,y]
      .VLine.II  :VLine   [x,y]
   .Overlay.III :Overlay
      .NdOverlay.I :NdOverlay   [veloity comp]
         :Curve   [time]   (v)
      .VLine.I     :VLine   [x,y]
      .VLine.II    :VLine   [x,y]
   .Overlay.IV  :Overlay
      .Curve.I  :Curve   [time]   (plasma_temperature)
      .VLine.I  :VLine   [x,y]
      .VLine.II :VLine   [x,y]

In [ ]:
from bokeh.io import output_notebook
output_notebook()